In [1]:
# IMPORTS
import typing as tp
import discogs_client as DiscogsClient
from rapidfuzz import process as r_process, fuzz as r_fuzz
from rich import print as rprint
import spotipy
from spotipy.oauth2 import SpotifyClientCredentials
import os
import json
import pathlib
import dotenv
env_file_name = '/../../../.env'
env_path = pathlib.Path(os.getcwd() + env_file_name)
if env_path.exists():
    dotenv.load_dotenv(env_path, override=True)
else:
    raise FileExistsError('env file does not exist')

SPOTIPY_CLIENT_ID = os.getenv('SPOTIFY_CLIENT_ID')
SPOTIPY_CLIENT_SECRET = os.getenv('SPOTIFY_CLIENT_SECRET')
DISCOGS_USER_TOKEN = os.getenv('DISCOGS_USER_TOKEN')

discogs_client = DiscogsClient.Client('ExampleApplication/0.1', user_token=DISCOGS_USER_TOKEN)
spotify_auth_manager = SpotifyClientCredentials(client_id=SPOTIPY_CLIENT_ID, client_secret=SPOTIPY_CLIENT_SECRET)
spotify_client = spotipy.Spotify(client_credentials_manager=spotify_auth_manager)



# DISCOGS

# SPOTIFY

# YOUTUBE

## Setup

In [ ]:
# Setup
# This does not work for some reason - the device connection thingy requires identifying the device
# Much check how to avoid that.

# from ytmusicapi import YTMusic, OAuthCredentials
# YTMUSIC_REFRESH_TOKEN = os.getenv('YTMUSIC_REFRESH_TOKEN')
# YTMUSIC_ACCESS_TOKEN = os.getenv('YTMUSIC_ACCESS_TOKEN')
# YTMUSIC_CLIENT_ID = os.getenv('YTMUSIC_CLIENT_ID')
# YTMUSIC_CLIENT_SECRET = os.getenv('YTMUSIC_CLIENT_SECRET')
# with open('../../oauth.json') as file:
#     oauth = json.load(file)
# oauth['access_token'] = YTMUSIC_ACCESS_TOKEN
# oauth['refresh_token'] = YTMUSIC_REFRESH_TOKEN
# oauth_credentials = OAuthCredentials(client_id=YTMUSIC_CLIENT_ID, client_secret=YTMUSIC_CLIENT_SECRET)
# yt = YTMusic(oauth, oauth_credentials=oauth_credentials)

# Alt-authorisation
# ytmusicapi.setup()

In [2]:
# Alternative
# Use the API unauthorised

from ytmusicapi import YTMusic
youtube_client = YTMusic()


## General info

Song and video are generally treated the same, but you can't add video to a playlist.

```python
search()
    browseId [artists, albums]
        get_artist
            ['albums']['browseId'] + ['albums']['params']
                get_artist_albums
                    [0]['browseId']
                        get_album
        get_album
```


In [ ]:
youtube_client.get_song('JxKjCOtRV_A')

In [23]:
browseId = get_artist_result['albums']['browseId']
params = get_artist_result['albums']['params']
get_artist_albums_result = youtube_client.get_artist_albums(browseId, params)

In [25]:
browseId = get_artist_albums_result[0]['browseId']
youtube_client.get_album(browseId)

{'title': 'CMBCRST',
 'type': 'Album',
 'thumbnails': [{'url': 'https://lh3.googleusercontent.com/wImnwfNF_NejEPxBssU0H-H7kwLNodzWqiVRUZxgUT9k7KVG1YGFTbEyBQTQlZWuhpnJTuJeBmcZLlABVA=w60-h60-l90-rj',
   'width': 60,
   'height': 60},
  {'url': 'https://lh3.googleusercontent.com/wImnwfNF_NejEPxBssU0H-H7kwLNodzWqiVRUZxgUT9k7KVG1YGFTbEyBQTQlZWuhpnJTuJeBmcZLlABVA=w120-h120-l90-rj',
   'width': 120,
   'height': 120},
  {'url': 'https://lh3.googleusercontent.com/wImnwfNF_NejEPxBssU0H-H7kwLNodzWqiVRUZxgUT9k7KVG1YGFTbEyBQTQlZWuhpnJTuJeBmcZLlABVA=w226-h226-l90-rj',
   'width': 226,
   'height': 226},
  {'url': 'https://lh3.googleusercontent.com/wImnwfNF_NejEPxBssU0H-H7kwLNodzWqiVRUZxgUT9k7KVG1YGFTbEyBQTQlZWuhpnJTuJeBmcZLlABVA=w544-h544-l90-rj',
   'width': 544,
   'height': 544}],
 'isExplicit': False,
 'description': None,
 'artists': [{'name': 'Combichrist', 'id': 'UCC4FxQ1RGHgC95YUh4nwD7w'}],
 'year': '2024',
 'trackCount': 23,
 'duration': '1 hour, 37 minutes',
 'audioPlaylistId': 'OLAK5uy_k

In [92]:
res = {
    'description': 'Combichrist is an American aggrotech / industrial metal band formed in 2003 by Norwegian vocalist Andy LaPlegua, who also founded the bands Icon of Coil, Panzer AG, Scandinavian Cock and Scandy. Combichrist was formed in Atlanta, Georgia. The band consists of only LaPlegua in the studio, with a changing lineup of live performing members. Releases by the band have been very successful in terms of sales and charted in top positions for dance and alternative charts worldwide.\n\nFrom Wikipedia (https://en.wikipedia.org/wiki/Combichrist) under Creative Commons Attribution CC-BY-SA 3.0 (http://creativecommons.org/licenses/by-sa/3.0/legalcode)',
    'views': '32,124,992 views',
    'name': 'CombiChrist',
    'channelId': 'UChkRHfQavQPQo2x5JkATA0w',
    'shuffleId': 'RDAOnD3IVfZiucPE13WeTAZ0Nw',
    'radioId': 'RDEMnD3IVfZiucPE13WeTAZ0Nw',
    'subscribers': '63.2K',
    'subscribed': False,
    'thumbnails': [
        {
            'url': 'https://lh3.googleusercontent.com/QvN2GI_GwaCFoZ1UuKVpAeOccis7Q02Lc5E5XRJ9l4rYxUa8bkmPEsJAhlqLxzkJWRFnNN7iX7-1Gdw=w540-h225-p-l90-rj',
            'width': 540,
            'height': 225},
            {'url': 'https://lh3.googleusercontent.com/QvN2GI_GwaCFoZ1UuKVpAeOccis7Q02Lc5E5XRJ9l4rYxUa8bkmPEsJAhlqLxzkJWRFnNN7iX7-1Gdw=w816-h340-p-l90-rj',
            'width': 816,
            'height': 340},
            {'url': 'https://lh3.googleusercontent.com/QvN2GI_GwaCFoZ1UuKVpAeOccis7Q02Lc5E5XRJ9l4rYxUa8bkmPEsJAhlqLxzkJWRFnNN7iX7-1Gdw=w1080-h450-p-l90-rj',
            'width': 1080,
            'height': 450
        }
    ],
    'songs': {
        'browseId': 'VLOLAK5uy_mp27YTeAbW2XS8AafI7K-3symK6w17YgA',
        'results': [
            {
                'videoId': 'JLp2e2c75C0',
                'title': 'Never Surrender',
                'artists': [{'name': 'Combichrist', 'id': 'UCC4FxQ1RGHgC95YUh4nwD7w'}],
                'album': {'name': 'Making Monsters', 'id': 'MPREb_I5yl3fkvCUu'},
                'likeStatus': 'INDIFFERENT',
                'inLibrary': None,
                'thumbnails': [
                    {
                        'url': 'https://lh3.googleusercontent.com/fw6ljhp3B3K4J-4xmspG9TvVuB_ET1RIOe3Wo4L14fwlxYBUB0mmbJWwrJBv_URwykZMSHDBtnEK-dcO=w60-h60-l90-rj',
                        'width': 60,
                        'height': 60
                    },
                    {
                        'url': 'https://lh3.googleusercontent.com/fw6ljhp3B3K4J-4xmspG9TvVuB_ET1RIOe3Wo4L14fwlxYBUB0mmbJWwrJBv_URwykZMSHDBtnEK-dcO=w120-h120-l90-rj',
                        'width': 120,
                        'height': 120
                    }
                ],
                'isAvailable': True,
                'isExplicit': False,
                'videoType': 'MUSIC_VIDEO_TYPE_ATV',
                'views': None
            },
            {
                'videoId': 'iV7VqOxZycM',
                'title': 'Throat Full of Glass',
                'artists': [{'name': 'Combichrist', 'id': 'UCC4FxQ1RGHgC95YUh4nwD7w'}],
                'album': {'name': 'Making Monsters', 'id': 'MPREb_I5yl3fkvCUu'},
                'likeStatus': 'INDIFFERENT',
                'inLibrary': None,
                'thumbnails': [{'url': 'https://lh3.googleusercontent.com/fw6ljhp3B3K4J-4xmspG9TvVuB_ET1RIOe3Wo4L14fwlxYBUB0mmbJWwrJBv_URwykZMSHDBtnEK-dcO=w60-h60-l90-rj',
                    'width': 60,
                    'height': 60},
                    {'url': 'https://lh3.googleusercontent.com/fw6ljhp3B3K4J-4xmspG9TvVuB_ET1RIOe3Wo4L14fwlxYBUB0mmbJWwrJBv_URwykZMSHDBtnEK-dcO=w120-h120-l90-rj',
                    'width': 120,
                    'height': 120}],
                'isAvailable': True,
                'isExplicit': False,
                'videoType': 'MUSIC_VIDEO_TYPE_ATV',
                'views': None
            },
            {
                'videoId': 'W5hqBefHG1k',
                'title': 'Sent to Destroy',
                'artists': [{'name': 'Combichrist', 'id': 'UCC4FxQ1RGHgC95YUh4nwD7w'}],
                'album': {'name': 'Today We Are All Demons', 'id': 'MPREb_pa0Asqgf0C7'},
                'likeStatus': 'INDIFFERENT',
                'inLibrary': None,
                'thumbnails': [{'url': 'https://lh3.googleusercontent.com/wFbBdbKJxuNSUET6ses0L95e_SGN3Im-dSZ8XdpPBrtxC4bn4LR_6kh1Gp5yE4dV-pIVcyK7wCkMDHPJ=w60-h60-l90-rj',
                    'width': 60,
                    'height': 60},
                    {'url': 'https://lh3.googleusercontent.com/wFbBdbKJxuNSUET6ses0L95e_SGN3Im-dSZ8XdpPBrtxC4bn4LR_6kh1Gp5yE4dV-pIVcyK7wCkMDHPJ=w120-h120-l90-rj',
                    'width': 120,
                    'height': 120}],
                'isAvailable': True,
                'isExplicit': False,
                'videoType': 'MUSIC_VIDEO_TYPE_ATV',
                'views': None
            },
            {
                'videoId': 'tt6yVYUWrYk',
                'title': 'Get Your Body Beat',
                'artists': [{'name': 'Combichrist', 'id': 'UCC4FxQ1RGHgC95YUh4nwD7w'}],
                'album': {'name': 'What the Fuck is Wrong With You People?',
                    'id': 'MPREb_n0JRn7cPNlJ'},
                'likeStatus': 'INDIFFERENT',
                'inLibrary': None,
                'thumbnails': [{'url': 'https://lh3.googleusercontent.com/AB-Hw38k9ID4WbQI2xNwZ3WGY8I61lp0xgHEiTfRh9cQTwFS7ubUCgRl2XvV6XQjHbFL5nhBzuqQuJEr=w60-h60-l90-rj',
                    'width': 60,
                    'height': 60},
                    {'url': 'https://lh3.googleusercontent.com/AB-Hw38k9ID4WbQI2xNwZ3WGY8I61lp0xgHEiTfRh9cQTwFS7ubUCgRl2XvV6XQjHbFL5nhBzuqQuJEr=w120-h120-l90-rj',
                    'width': 120,
                    'height': 120}],
                'isAvailable': True,
                'isExplicit': True,
                'videoType': 'MUSIC_VIDEO_TYPE_ATV',
                'views': None
            },
            {
                'videoId': 'EaRzCpfup3w',
                'title': 'Gotta Go',
                'artists': [{'name': 'Combichrist', 'id': 'UCC4FxQ1RGHgC95YUh4nwD7w'}],
                'album': {'name': 'No Redemption (Official DMC Devil May Cry Soundtrack)',
                    'id': 'MPREb_awECvwj6uRf'},
                'likeStatus': 'INDIFFERENT',
                'inLibrary': None,
                'thumbnails': [{'url': 'https://lh3.googleusercontent.com/FkpgITfJvQ_AF7C3cl9T-iJ97EltVM43o6zka53MXLUS1puUUohn7GKsJpHL5iIlLuEKCGi1JWl3A0o=w60-h60-l90-rj',
                    'width': 60,
                    'height': 60},
                    {'url': 'https://lh3.googleusercontent.com/FkpgITfJvQ_AF7C3cl9T-iJ97EltVM43o6zka53MXLUS1puUUohn7GKsJpHL5iIlLuEKCGi1JWl3A0o=w120-h120-l90-rj',
                    'width': 120,
                    'height': 120}],
                'isAvailable': True,
                'isExplicit': False,
                'videoType': 'MUSIC_VIDEO_TYPE_ATV',
                'views': None
            }
        ]
    },
    'albums': {
        'browseId': 'MPADUCC4FxQ1RGHgC95YUh4nwD7w',
        'results': [
            {
                'title': 'CMBCRST',
                'type': '2024',
                'artists': [],
                'browseId': 'MPREb_N62W2TlZYuZ',
                'audioPlaylistId': 'OLAK5uy_kVNFc3M-jx1PPFShyAfu2qxw6QBYGaekw',
                'thumbnails': [{'url': 'https://lh3.googleusercontent.com/wImnwfNF_NejEPxBssU0H-H7kwLNodzWqiVRUZxgUT9k7KVG1YGFTbEyBQTQlZWuhpnJTuJeBmcZLlABVA=w226-h226-l90-rj',
                    'width': 226,
                    'height': 226},
                    {'url': 'https://lh3.googleusercontent.com/wImnwfNF_NejEPxBssU0H-H7kwLNodzWqiVRUZxgUT9k7KVG1YGFTbEyBQTQlZWuhpnJTuJeBmcZLlABVA=w544-h544-l90-rj',
                    'width': 544,
                    'height': 544}],
                'isExplicit': True
            },
            {
                'title': 'One Fire',
                'type': '2019',
                'artists': [],
                'browseId': 'MPREb_TJBd7TTFTmi',
                'audioPlaylistId': 'OLAK5uy_lLVBbfrL31cFotsw_BALDb2oWJNJiPZWI',
                'thumbnails': [{'url': 'https://lh3.googleusercontent.com/FiVqqsVtSigx87Mmix3J0fcp7kRLMDPbVZs34iBEduwqsg1Z6koJ--7VPxn3NLQQNokEEBjSfVSg_GE=w226-h226-l90-rj',
                    'width': 226,
                    'height': 226},
                    {'url': 'https://lh3.googleusercontent.com/FiVqqsVtSigx87Mmix3J0fcp7kRLMDPbVZs34iBEduwqsg1Z6koJ--7VPxn3NLQQNokEEBjSfVSg_GE=w544-h544-l90-rj',
                    'width': 544,
                    'height': 544}],
                'isExplicit': True
            },
            {
                'title': 'This Is Where Death Begins',
                'type': '2016',
                'artists': [],
                'browseId': 'MPREb_CNGBRPgI2zL',
                'audioPlaylistId': 'OLAK5uy_kcV3ney03XbYXJB5eG5nrimKlMpKGEA_U',
                'thumbnails': [{'url': 'https://lh3.googleusercontent.com/-micPgattM9indEK8_gjgSd5zStIJnWqZrKiD5X8tpycue7BaB2RzFGu7GLqfgMuiUqnLCyRUh5UXpNUFQ=w226-h226-l90-rj',
                    'width': 226,
                    'height': 226},
                    {'url': 'https://lh3.googleusercontent.com/-micPgattM9indEK8_gjgSd5zStIJnWqZrKiD5X8tpycue7BaB2RzFGu7GLqfgMuiUqnLCyRUh5UXpNUFQ=w544-h544-l90-rj',
                    'width': 544,
                    'height': 544}],
                'isExplicit': True
            },
            {
                'title': 'We Love You (Deluxe Edition)',
                'type': '2014',
                'artists': [],
                'browseId': 'MPREb_6lqmRjbdIts',
                'audioPlaylistId': 'OLAK5uy_mrICrAqGZR9TfLl0A-XCAOKMyoBdutem8',
                'thumbnails': [{'url': 'https://lh3.googleusercontent.com/mW1l4jkjcUH99AJfnAT0rhKwbtR5ebrJVoCxKT0Q2K0xkM9LywDLmxm_UyLC7qi0bRJoufrcEbIORarB9A=w226-h226-l90-rj',
                    'width': 226,
                    'height': 226},
                    {'url': 'https://lh3.googleusercontent.com/mW1l4jkjcUH99AJfnAT0rhKwbtR5ebrJVoCxKT0Q2K0xkM9LywDLmxm_UyLC7qi0bRJoufrcEbIORarB9A=w544-h544-l90-rj',
                    'width': 544,
                    'height': 544}],
                'isExplicit': True
            },
            {
                'title': 'No Redemption (Official DMC Devil May Cry Soundtrack)',
                'type': '2012',
                'artists': [],
                'browseId': 'MPREb_awECvwj6uRf',
                'audioPlaylistId': 'OLAK5uy_mDa60eITWbd-spUMiJB-4vJcf714fxZt4',
                'thumbnails': [{'url': 'https://lh3.googleusercontent.com/FkpgITfJvQ_AF7C3cl9T-iJ97EltVM43o6zka53MXLUS1puUUohn7GKsJpHL5iIlLuEKCGi1JWl3A0o=w226-h226-l90-rj',
                    'width': 226,
                    'height': 226},
                    {'url': 'https://lh3.googleusercontent.com/FkpgITfJvQ_AF7C3cl9T-iJ97EltVM43o6zka53MXLUS1puUUohn7GKsJpHL5iIlLuEKCGi1JWl3A0o=w544-h544-l90-rj',
                    'width': 544,
                    'height': 544}],
                'isExplicit': False
            },
            {
                'title': 'Making Monsters',
                'type': '2010',
                'artists': [],
                'browseId': 'MPREb_I5yl3fkvCUu',
                'audioPlaylistId': 'OLAK5uy_nfR7YX2mSNSIlHvoD6Vss761ZSbqtEyZM',
                'thumbnails': [{'url': 'https://lh3.googleusercontent.com/fw6ljhp3B3K4J-4xmspG9TvVuB_ET1RIOe3Wo4L14fwlxYBUB0mmbJWwrJBv_URwykZMSHDBtnEK-dcO=w226-h226-l90-rj',
                    'width': 226,
                    'height': 226},
                    {'url': 'https://lh3.googleusercontent.com/fw6ljhp3B3K4J-4xmspG9TvVuB_ET1RIOe3Wo4L14fwlxYBUB0mmbJWwrJBv_URwykZMSHDBtnEK-dcO=w544-h544-l90-rj',
                    'width': 544,
                    'height': 544}],
                'isExplicit': True
            },
            {
                'title': 'Scarred',
                'type': '2010',
                'artists': [],
                'browseId': 'MPREb_UY42LKK8GrH',
                'audioPlaylistId': 'OLAK5uy_l0cL1b4aHiUQ58S3ALsL-aMc1xLFHyeGE',
                'thumbnails': [{'url': 'https://lh3.googleusercontent.com/D07W1RtTEbJUSzkzVbaE8cpecQ9HrsHLnR9vG4eW6cONsgfq84_Fs2drQCfeNxbXnMfBqUGUcYNKyLrN=w226-h226-l90-rj',
                    'width': 226,
                    'height': 226},
                    {'url': 'https://lh3.googleusercontent.com/D07W1RtTEbJUSzkzVbaE8cpecQ9HrsHLnR9vG4eW6cONsgfq84_Fs2drQCfeNxbXnMfBqUGUcYNKyLrN=w544-h544-l90-rj',
                    'width': 544,
                    'height': 544}],
                'isExplicit': False
            },
            {
                'title': 'Today We Are All Demons',
                'type': '2009',
                'artists': [],
                'browseId': 'MPREb_pa0Asqgf0C7',
                'audioPlaylistId': 'OLAK5uy_mifATcmEJhV_33GNNb9oz22mw49n6Fh9Q',
                'thumbnails': [{'url': 'https://lh3.googleusercontent.com/wFbBdbKJxuNSUET6ses0L95e_SGN3Im-dSZ8XdpPBrtxC4bn4LR_6kh1Gp5yE4dV-pIVcyK7wCkMDHPJ=w226-h226-l90-rj',
                    'width': 226,
                    'height': 226},
                    {'url': 'https://lh3.googleusercontent.com/wFbBdbKJxuNSUET6ses0L95e_SGN3Im-dSZ8XdpPBrtxC4bn4LR_6kh1Gp5yE4dV-pIVcyK7wCkMDHPJ=w544-h544-l90-rj',
                    'width': 544,
                    'height': 544}],
                'isExplicit': False
            },
            {
                'title': 'What the F^^k Is Wrong with You People? - Dark Side',
                'type': '2007',
                'artists': [],
                'browseId': 'MPREb_ZNTLqpn0xTj',
                'audioPlaylistId': 'OLAK5uy_nwsbNxJnwq4cZJJ7kiFTqNYToTq6GumVM',
                'thumbnails': [{'url': 'https://lh3.googleusercontent.com/4AzmZAcs__ysNXBhLNGeUVBCNp7F3pry5McznvRdWQb6l_G_zYdbEllfj8L8WGTA_HwNww372AKxn3Sz8g=w226-h226-l90-rj',
                    'width': 226,
                    'height': 226},
                    {'url': 'https://lh3.googleusercontent.com/4AzmZAcs__ysNXBhLNGeUVBCNp7F3pry5McznvRdWQb6l_G_zYdbEllfj8L8WGTA_HwNww372AKxn3Sz8g=w544-h544-l90-rj',
                    'width': 544,
                    'height': 544}],
                'isExplicit': True
            },
            {
                'title': 'What the Fuck is Wrong With You People?',
                'type': '2007',
                'artists': [],
                'browseId': 'MPREb_n0JRn7cPNlJ',
                'audioPlaylistId': 'OLAK5uy_kPNB38YcEhhnGcLDrTouAzcvWbZmbwLno',
                'thumbnails': [{'url': 'https://lh3.googleusercontent.com/AB-Hw38k9ID4WbQI2xNwZ3WGY8I61lp0xgHEiTfRh9cQTwFS7ubUCgRl2XvV6XQjHbFL5nhBzuqQuJEr=w226-h226-l90-rj',
                    'width': 226,
                    'height': 226},
                    {'url': 'https://lh3.googleusercontent.com/AB-Hw38k9ID4WbQI2xNwZ3WGY8I61lp0xgHEiTfRh9cQTwFS7ubUCgRl2XvV6XQjHbFL5nhBzuqQuJEr=w544-h544-l90-rj',
                    'width': 544,
                    'height': 544}],
                'isExplicit': True
            }
        ],
        'params': 'ggMIegYIARoCAQI%3D'
    },
    'singles': {
        'browseId': 'MPADUCC4FxQ1RGHgC95YUh4nwD7w',
        'results': [
            {
                'title': 'Violence Solves Everything PT II (The end of a dream)',
                'year': 'EP',
                'browseId': 'MPREb_OttDNUgeAFj',
                'thumbnails': [{'url': 'https://lh3.googleusercontent.com/xljHZDwlpSfgTW_xu_95Bx70idWp4oYz16Stb8KanyXA9cn5g4zXd3QfaMl1j2FcibXUSFg5OaDz0sg=w226-h226-l90-rj',
                    'width': 226,
                    'height': 226},
                    {'url': 'https://lh3.googleusercontent.com/xljHZDwlpSfgTW_xu_95Bx70idWp4oYz16Stb8KanyXA9cn5g4zXd3QfaMl1j2FcibXUSFg5OaDz0sg=w544-h544-l90-rj',
                    'width': 544,
                    'height': 544}]
            },
            {
                    'title': 'Planet Doom',
                'year': 'EP',
                'browseId': 'MPREb_rRFzc4QPGku',
                'thumbnails': [{'url': 'https://lh3.googleusercontent.com/chtTeGEPwCu_2LPlJbJKIvV1WhtT5NTiQzNavOJkrtzO2Wx-mqcQ5eirmhObfJUULQvGJVCVp0jLlPaT=w226-h226-l90-rj',
                    'width': 226,
                    'height': 226},
                    {'url': 'https://lh3.googleusercontent.com/chtTeGEPwCu_2LPlJbJKIvV1WhtT5NTiQzNavOJkrtzO2Wx-mqcQ5eirmhObfJUULQvGJVCVp0jLlPaT=w544-h544-l90-rj',
                    'width': 544,
                    'height': 544}]
            },
            {
                    'title': 'Heads Off',
                'year': 'EP',
                'browseId': 'MPREb_Jm8fCzRYyNe',
                'thumbnails': [{'url': 'https://lh3.googleusercontent.com/-kQCVCLbt7DiMQbcvIphrNEMiyPNKya8UbWJ-vJixbuMEK3ip54nvNlP5G0133DuvqMuHHqaMmnMhkHf=w226-h226-l90-rj',
                    'width': 226,
                    'height': 226},
                    {'url': 'https://lh3.googleusercontent.com/-kQCVCLbt7DiMQbcvIphrNEMiyPNKya8UbWJ-vJixbuMEK3ip54nvNlP5G0133DuvqMuHHqaMmnMhkHf=w544-h544-l90-rj',
                    'width': 544,
                    'height': 544}]
            },
            {
                    'title': 'Modern Demon',
                'year': 'Single',
                'browseId': 'MPREb_z8CNT0oZeHs',
                'thumbnails': [{'url': 'https://lh3.googleusercontent.com/24EPCulA2L2odytmcRmOBToGjO_t1oVld-dCTRviBsAkD6bPNk6JKTB4wSXkS-PnCALVIOIwZEBemfew=w226-h226-l90-rj',
                    'width': 226,
                    'height': 226},
                    {'url': 'https://lh3.googleusercontent.com/24EPCulA2L2odytmcRmOBToGjO_t1oVld-dCTRviBsAkD6bPNk6JKTB4wSXkS-PnCALVIOIwZEBemfew=w544-h544-l90-rj',
                    'width': 544,
                    'height': 544}]
            },
            {
                    'title': 'Compliance',
                'year': 'Single',
                'browseId': 'MPREb_h470C4za36N',
                'thumbnails': [{'url': 'https://lh3.googleusercontent.com/nleqHHiouy2GG-AIKTAqtrtsbX2sUdjtnz1TIl5n-wioA2gssr8WI9vPE_tcrwCKxM1-TTI_qvch78YA=w226-h226-l90-rj',
                    'width': 226,
                    'height': 226},
                    {'url': 'https://lh3.googleusercontent.com/nleqHHiouy2GG-AIKTAqtrtsbX2sUdjtnz1TIl5n-wioA2gssr8WI9vPE_tcrwCKxM1-TTI_qvch78YA=w544-h544-l90-rj',
                    'width': 544,
                    'height': 544}]
            },
            {
                    'title': 'Not My Enemy',
                'year': 'Single',
                'browseId': 'MPREb_kG1D2i1pKaq',
                'thumbnails': [{'url': 'https://lh3.googleusercontent.com/Wu-Wf1GRWTNTYPeIJe62K_mydMBnzaxzZ83dxEck2YWzILuZ3nOaQDCF2gP7NWOXie0wkeKvExanI8JXCg=w226-h226-l90-rj',
                    'width': 226,
                    'height': 226},
                    {'url': 'https://lh3.googleusercontent.com/Wu-Wf1GRWTNTYPeIJe62K_mydMBnzaxzZ83dxEck2YWzILuZ3nOaQDCF2gP7NWOXie0wkeKvExanI8JXCg=w544-h544-l90-rj',
                    'width': 544,
                    'height': 544}]
            },
            {
                    'title': 'Understand',
                'year': 'Single',
                'browseId': 'MPREb_qUYFZcXl5yt',
                'thumbnails': [{'url': 'https://lh3.googleusercontent.com/BbttBxTFEP52jHZ7QLrIYYKEW0pRcVcXZiQB6X2D4OmFZJiJQ0_Ep7vytXgT8g2-SZAlBqz716G-Q6kVEA=w226-h226-l90-rj',
                    'width': 226,
                    'height': 226},
                    {'url': 'https://lh3.googleusercontent.com/BbttBxTFEP52jHZ7QLrIYYKEW0pRcVcXZiQB6X2D4OmFZJiJQ0_Ep7vytXgT8g2-SZAlBqz716G-Q6kVEA=w544-h544-l90-rj',
                    'width': 544,
                    'height': 544}]
            },
            {
                    'title': 'Guns at Last Dawn (feat. Burton C. Bell)',
                'year': 'Single',
                'browseId': 'MPREb_MkTBRwA4Rye',
                'thumbnails': [{'url': 'https://lh3.googleusercontent.com/Wu-FbZ60U0ko5vYvBsha3WIR5PMBKSsdti8i05dXwEgCuL9HutpLbVF5oour6slCxaSRpCREGdDwVI1i=w226-h226-l90-rj',
                    'width': 226,
                    'height': 226},
                    {'url': 'https://lh3.googleusercontent.com/Wu-FbZ60U0ko5vYvBsha3WIR5PMBKSsdti8i05dXwEgCuL9HutpLbVF5oour6slCxaSRpCREGdDwVI1i=w544-h544-l90-rj',
                    'width': 544,
                    'height': 544}]
            },
            {
                    'title': 'Hate Like Me',
                'year': 'Single',
                'browseId': 'MPREb_QG58iRjNIkP',
                'thumbnails': [{'url': 'https://lh3.googleusercontent.com/ZvefnbYyBpOKewsBvXYRdYmP17AyRZjhsJ1HXgxCMRC1kXToS9EN_xUEN2V6blE4cOJQBj-ybtij70Ab=w226-h226-l90-rj',
                    'width': 226,
                    'height': 226},
                    {'url': 'https://lh3.googleusercontent.com/ZvefnbYyBpOKewsBvXYRdYmP17AyRZjhsJ1HXgxCMRC1kXToS9EN_xUEN2V6blE4cOJQBj-ybtij70Ab=w544-h544-l90-rj',
                    'width': 544,
                    'height': 544}]
            },
            {
                    'title': 'Broken : United',
                'year': 'Single',
                'browseId': 'MPREb_FUD2xtxEhg1',
                'thumbnails': [{'url': 'https://lh3.googleusercontent.com/o4BFqzvKYFdc2eVhKrDdZnSHRbjU_O1nLDETNakZBEmhmBFKMkbKG1jwzxsjJLC6KWO8ncXUy5gyVBs96A=w226-h226-l90-rj',
                    'width': 226,
                    'height': 226},
                    {'url': 'https://lh3.googleusercontent.com/o4BFqzvKYFdc2eVhKrDdZnSHRbjU_O1nLDETNakZBEmhmBFKMkbKG1jwzxsjJLC6KWO8ncXUy5gyVBs96A=w544-h544-l90-rj',
                    'width': 544,
                    'height': 544}]
            }
        ],
        'params': 'ggMIegYIAhoCAQI%3D'
    },
    'videos': {
        'browseId': 'VLOLAK5uy_mRLZT88KMumMFegjVHBvMQI-3C5v54pXY',
        'results': [
            {
                'title': 'Sent to Destroy',
                'videoId': 'A8pBe1Zbd3Y',
                'artists': [{'name': 'CombiChrist', 'id': 'UCC4FxQ1RGHgC95YUh4nwD7w'}],
                'playlistId': 'OLAK5uy_mRLZT88KMumMFegjVHBvMQI-3C5v54pXY',
                'thumbnails': [{'url': 'https://i.ytimg.com/vi/A8pBe1Zbd3Y/hqdefault.jpg?sqp=-oaymwEWCMACELQBIAQqCghQEJADGFogjgJIWg&rs=AMzJL3lK6UhjeAt7AIj024WdkuYNOGKmgQ',
                    'width': 320,
                    'height': 180}],
                'views': '3.5M'
            },
            {
                'title': 'Get Your Body Beat',
                'videoId': 'oExRglZ_06Q',
                'artists': [{'name': 'CombiChrist', 'id': 'UCC4FxQ1RGHgC95YUh4nwD7w'}],
                'playlistId': 'OLAK5uy_mRLZT88KMumMFegjVHBvMQI-3C5v54pXY',
                'thumbnails': [{'url': 'https://i.ytimg.com/vi/oExRglZ_06Q/hqdefault.jpg?sqp=-oaymwEWCMACELQBIAQqCghQEJADGFogjgJIWg&rs=AMzJL3lAwYfsfiHs8754Fx0A6R46hnHojg',
                    'width': 320,
                    'height': 180}],
                'views': '817K'
            },
             
                {'title': 'Modern Demon',
                'videoId': 'aSlXqRXTE9M',
                'artists': [{'name': 'CombiChrist', 'id': 'UCC4FxQ1RGHgC95YUh4nwD7w'}],
                'playlistId': 'OLAK5uy_mRLZT88KMumMFegjVHBvMQI-3C5v54pXY',
                'thumbnails': [{'url': 'https://i.ytimg.com/vi/aSlXqRXTE9M/sddefault.jpg?sqp=-oaymwEWCJADEOEBIAQqCghqEJQEGHgg6AJIWg&rs=AMzJL3kwnQP0V8Kf5a4JY6mvnuGDB9HwoA',
                    'width': 400,
                    'height': 225},
                    {'url': 'https://i.ytimg.com/vi/aSlXqRXTE9M/hq720.jpg?sqp=-oaymwEXCKAGEMIDIAQqCwjVARCqCBh4INgESFo&rs=AMzJL3mYtGEwcRPfvfU79etSHVJYBedmCQ',
                    'width': 800,
                    'height': 450}],
                'views': '213K'
            },
             
                {'title': 'My Life My Rules',
                'videoId': '0tEUP7vDMbo',
                'artists': [{'name': 'CombiChrist', 'id': 'UCC4FxQ1RGHgC95YUh4nwD7w'}],
                'playlistId': 'OLAK5uy_mRLZT88KMumMFegjVHBvMQI-3C5v54pXY',
                'thumbnails': [{'url': 'https://i.ytimg.com/vi/0tEUP7vDMbo/sddefault.jpg?sqp=-oaymwEWCJADEOEBIAQqCghqEJQEGHgg6AJIWg&rs=AMzJL3kfW-gGgJ5nfeCJDqpAr-wtmq6Z3A',
                    'width': 400,
                    'height': 225},
                    {'url': 'https://i.ytimg.com/vi/0tEUP7vDMbo/hq720.jpg?sqp=-oaymwEXCKAGEMIDIAQqCwjVARCqCBh4INgESFo&rs=AMzJL3mEsBhLRf-a59Rrs-hzfQIQty_FGQ',
                    'width': 800,
                    'height': 450}],
                'views': '2.4M'
            },
             
                {'title': 'Hate Like Me',
                'videoId': 'cGY6M59IiS0',
                'artists': [{'name': 'CombiChrist', 'id': 'UCC4FxQ1RGHgC95YUh4nwD7w'}],
                'playlistId': 'OLAK5uy_mRLZT88KMumMFegjVHBvMQI-3C5v54pXY',
                'thumbnails': [{'url': 'https://i.ytimg.com/vi/cGY6M59IiS0/sddefault.jpg?sqp=-oaymwEWCJADEOEBIAQqCghqEJQEGHgg6AJIWg&rs=AMzJL3kkKdwLUr1ZeEq2blyxX4EUSazfjA',
                    'width': 400,
                    'height': 225},
                    {'url': 'https://i.ytimg.com/vi/cGY6M59IiS0/hq720.jpg?sqp=-oaymwEXCKAGEMIDIAQqCwjVARCqCBh4INgESFo&rs=AMzJL3nAROnNJM4K-snmmLTktc8uw58iCA',
                    'width': 800,
                    'height': 450}],
                'views': '623K'
            },
             
                {'title': 'Heads Off',
                'videoId': 'sOd7SOZTMsU',
                'artists': [{'name': 'CombiChrist', 'id': 'UCC4FxQ1RGHgC95YUh4nwD7w'}],
                'playlistId': 'OLAK5uy_mRLZT88KMumMFegjVHBvMQI-3C5v54pXY',
                'thumbnails': [{'url': 'https://i.ytimg.com/vi/sOd7SOZTMsU/sddefault.jpg?sqp=-oaymwEWCJADEOEBIAQqCghqEJQEGHgg6AJIWg&rs=AMzJL3kNg_owtwwubI-zEuUrBGtm1tQ4KQ',
                    'width': 400,
                    'height': 225},
                    {'url': 'https://i.ytimg.com/vi/sOd7SOZTMsU/hq720.jpg?sqp=-oaymwEXCKAGEMIDIAQqCwjVARCqCBh4INgESFo&rs=AMzJL3nekamjpe2wvFH8f2SpLqwi8yIvHg',
                    'width': 800,
                    'height': 450}],
                'views': '272K'
            },
             
                {'title': 'From My Cold Dead Hands',
                'videoId': 'E9Faf4MlbDk',
                'artists': [{'name': 'CombiChrist', 'id': 'UCC4FxQ1RGHgC95YUh4nwD7w'}],
                'playlistId': 'OLAK5uy_mRLZT88KMumMFegjVHBvMQI-3C5v54pXY',
                'thumbnails': [{'url': 'https://i.ytimg.com/vi/E9Faf4MlbDk/sddefault.jpg?sqp=-oaymwEWCJADEOEBIAQqCghqEJQEGHgg6AJIWg&rs=AMzJL3mquUgin_0_7F_iHyZSfrIzjdfEoA',
                    'width': 400,
                    'height': 225},
                    {'url': 'https://i.ytimg.com/vi/E9Faf4MlbDk/hq720.jpg?sqp=-oaymwEXCKAGEMIDIAQqCwjVARCqCBh4INgESFo&rs=AMzJL3klB1ubZP7R5MpR-j5xy91bbm28Vg',
                    'width': 800,
                    'height': 450}],
                'views': '851K'
            },
             
                {'title': 'Skullcrusher',
                'videoId': 'FWCw0blnhN8',
                'artists': [{'name': 'CombiChrist', 'id': 'UCC4FxQ1RGHgC95YUh4nwD7w'}],
                'playlistId': 'OLAK5uy_mRLZT88KMumMFegjVHBvMQI-3C5v54pXY',
                'thumbnails': [{'url': 'https://i.ytimg.com/vi/FWCw0blnhN8/sddefault.jpg?sqp=-oaymwEWCJADEOEBIAQqCghqEJQEGHgg6AJIWg&rs=AMzJL3kuDDOS8Qy6Nk3mPqpat7U67lFmZw',
                    'width': 400,
                    'height': 225},
                    {'url': 'https://i.ytimg.com/vi/FWCw0blnhN8/hq720.jpg?sqp=-oaymwEXCKAGEMIDIAQqCwjVARCqCBh4INgESFo&rs=AMzJL3nORTzKAfiVFvvjQ5C6QoYV_Mpfqw',
                    'width': 800,
                    'height': 450}],
                'views': '1.2M'
            },
             
                {'title': 'Not My Enemy',
                'videoId': 'nyzue_ZCXZc',
                'artists': [{'name': 'CombiChrist', 'id': 'UCC4FxQ1RGHgC95YUh4nwD7w'}],
                'playlistId': 'OLAK5uy_mRLZT88KMumMFegjVHBvMQI-3C5v54pXY',
                'thumbnails': [{'url': 'https://i.ytimg.com/vi/nyzue_ZCXZc/sddefault.jpg?sqp=-oaymwEWCJADEOEBIAQqCghqEJQEGHgg6AJIWg&rs=AMzJL3kLernap4j5dyWhO3l7wjht_zz1Hg',
                    'width': 400,
                    'height': 225},
                    {'url': 'https://i.ytimg.com/vi/nyzue_ZCXZc/hq720.jpg?sqp=-oaymwEXCKAGEMIDIAQqCwjVARCqCBh4INgESFo&rs=AMzJL3k2A1O0dkqRU3erx-YCJgGR7YEHhQ',
                    'width': 800,
                    'height': 450}],
                'views': '296K'
            },
             
                {'title': 'Combichrist - Deathbed (music video)',
                'videoId': '8N0PVjS3bMA',
                'artists': [{'name': 'CombiChrist', 'id': 'UCC4FxQ1RGHgC95YUh4nwD7w'}],
                'playlistId': 'OLAK5uy_mRLZT88KMumMFegjVHBvMQI-3C5v54pXY',
                'thumbnails': [{'url': 'https://i.ytimg.com/vi/8N0PVjS3bMA/hqdefault.jpg?sqp=-oaymwEWCMACELQBIAQqCghQEJADGFogjgJIWg&rs=AMzJL3k65sUCFGcKklskzR106WXXQ-raDA',
                    'width': 320,
                    'height': 180}],
                'views': '298K'}],
                'params': 'ggMCCAI%3D'},
                'related': {'browseId': None,
                'results': [{'title': '3Teeth',
                'browseId': 'UC5zWjQAxayKEJnbxbSuR7ug',
                'subscribers': '78.3K',
                'thumbnails': [{'url': 'https://lh3.googleusercontent.com/Hw552vr5-ieeH3wyTtA5tVuCoGSzcnM-RWtonLW8KO_G9XMbOzdpfg76h1C3xofOnS4Ki2RPPs7xMCM=w226-h226-p-l90-rj',
                    'width': 226,
                    'height': 226},
                    {'url': 'https://lh3.googleusercontent.com/Hw552vr5-ieeH3wyTtA5tVuCoGSzcnM-RWtonLW8KO_G9XMbOzdpfg76h1C3xofOnS4Ki2RPPs7xMCM=w544-h544-p-l90-rj',
                    'width': 544,
                    'height': 544}]
                },
                {
                    'title': 'Faderhead',
                'browseId': 'UCnsZHT0EGeqeTzN9OR2uQqw',
                'subscribers': '27.8K',
                'thumbnails': [{'url': 'https://lh3.googleusercontent.com/DxA3r_Vz39qp4Ouy4hdoF4_TcWbqH36oECwkmNhhpmIfGsL7BVj33UaA0snnwbFrjmXvs3yaX2UJMg=w226-h226-p-l90-rj',
                    'width': 226,
                    'height': 226},
                    {'url': 'https://lh3.googleusercontent.com/DxA3r_Vz39qp4Ouy4hdoF4_TcWbqH36oECwkmNhhpmIfGsL7BVj33UaA0snnwbFrjmXvs3yaX2UJMg=w544-h544-p-l90-rj',
                    'width': 544,
                    'height': 544}]
                },
                {
                    'title': 'Aesthetic Perfection',
                'browseId': 'UCDYJdjGXXUm23JEjtr8Fqbw',
                'subscribers': '56K',
                'thumbnails': [{'url': 'https://lh3.googleusercontent.com/a-/ALV-UjWgma0Wpg-3gq73VlslGrEQXHIJDDjvWuGI_PDckCnYLfWSFzBf=w226-h226-l90-rj',
                    'width': 226,
                    'height': 226},
                    {'url': 'https://lh3.googleusercontent.com/a-/ALV-UjWgma0Wpg-3gq73VlslGrEQXHIJDDjvWuGI_PDckCnYLfWSFzBf=w544-h544-l90-rj',
                    'width': 544,
                    'height': 544}]
                },
                {
                    'title': 'Grendel',
                'browseId': 'UCFeXJhNBcNWydmK_PRRSB-A',
                'subscribers': '5.91K',
                'thumbnails': [{'url': 'https://lh3.googleusercontent.com/ALMN78i7pf_vX6CqT_DOAdTDEIhwicr9LMiUPIAPkno0Lwt6sP8HU4vNy79wLquCwFBdoidE8Z8vFeq93Q=w226-h226-l90-rj',
                    'width': 226,
                    'height': 226},
                    {'url': 'https://lh3.googleusercontent.com/ALMN78i7pf_vX6CqT_DOAdTDEIhwicr9LMiUPIAPkno0Lwt6sP8HU4vNy79wLquCwFBdoidE8Z8vFeq93Q=w544-h544-l90-rj',
                    'width': 544,
                    'height': 544}]
                },
                {
                    'title': 'Hanzel und Gretyl',
                'browseId': 'UCLf-L-pOXzHWdE-TgtYn0kg',
                'subscribers': '8.18K',
                'thumbnails': [{'url': 'https://lh3.googleusercontent.com/DZdBfuA-p1YKtk52WdpNI5gLN742lFfKlGaJ0nZJRZN_TCmnYeQxZX893gUmMD5cATlW0L2yYJi51WQ=w226-h226-l90-rj',
                    'width': 226,
                    'height': 226},
                    {'url': 'https://lh3.googleusercontent.com/DZdBfuA-p1YKtk52WdpNI5gLN742lFfKlGaJ0nZJRZN_TCmnYeQxZX893gUmMD5cATlW0L2yYJi51WQ=w544-h544-l90-rj',
                    'width': 544,
                    'height': 544}]
                },
                {
                    'title': 'Ludovico Technique',
                'browseId': 'UCcrpO0YWGs_Z7DR_XaVEDtw',
                'subscribers': '7.32K',
                'thumbnails': [{'url': 'https://lh3.googleusercontent.com/Umlg5ob1874oD_dnREl25Mh30kd2GXoLSGE6oJvqCo36cI9I86BU1ri2QAGZDuN3EtId60A_vwuVS4Q=w226-h226-l90-rj',
                    'width': 226,
                    'height': 226},
                    {'url': 'https://lh3.googleusercontent.com/Umlg5ob1874oD_dnREl25Mh30kd2GXoLSGE6oJvqCo36cI9I86BU1ri2QAGZDuN3EtId60A_vwuVS4Q=w544-h544-l90-rj',
                    'width': 544,
                    'height': 544}]
                },
                {
                    'title': 'SKOLD',
                'browseId': 'UCp4GefKBXghXR63olX3movQ',
                'subscribers': '6.24K',
                'thumbnails': [{'url': 'https://lh3.googleusercontent.com/4sxMAuwHVbSR1mQtyt2otjeQm9X8RxZNuleOtjM8MQu6dZ67S_A5CjEHEpszGVDyep8OTNxvowKcMA=w226-h226-p-l90-rj',
                    'width': 226,
                    'height': 226},
                    {'url': 'https://lh3.googleusercontent.com/4sxMAuwHVbSR1mQtyt2otjeQm9X8RxZNuleOtjM8MQu6dZ67S_A5CjEHEpszGVDyep8OTNxvowKcMA=w544-h544-p-l90-rj',
                    'width': 544,
                    'height': 544}]
                },
                {
                    'title': 'Suicide Commando',
                'browseId': 'UCrsvJ1npu-ox0CnncGhq2QA',
                'subscribers': '13.5K',
                'thumbnails': [{'url': 'https://lh3.googleusercontent.com/SLbQJSEAhMXuol3I7vxWHKPyTH0WqWs9eyLJocoKGdEGEhYyUStDylIvwkqJry0U3xXKcReo8utR1A4=w226-h226-l90-rj',
                    'width': 226,
                    'height': 226},
                    {'url': 'https://lh3.googleusercontent.com/SLbQJSEAhMXuol3I7vxWHKPyTH0WqWs9eyLJocoKGdEGEhYyUStDylIvwkqJry0U3xXKcReo8utR1A4=w544-h544-l90-rj',
                    'width': 544,
                    'height': 544}]
                },
                {
                    'title': 'Psyclon Nine',
                'browseId': 'UCc8jU5W93y3clpohno9e-Gw',
                'subscribers': '9.37K',
                'thumbnails': [{'url': 'https://lh3.googleusercontent.com/PtvAtCsiE_8t55x23X8QDcy5eXYgIQXYCEIkpb8eCIwIkjyjNRhqh3wQ3J0R8Nd6T7PNu_n0vhmFpeDs=w226-h226-l90-rj',
                    'width': 226,
                    'height': 226},
                    {'url': 'https://lh3.googleusercontent.com/PtvAtCsiE_8t55x23X8QDcy5eXYgIQXYCEIkpb8eCIwIkjyjNRhqh3wQ3J0R8Nd6T7PNu_n0vhmFpeDs=w544-h544-l90-rj',
                    'width': 544,
                    'height': 544}]
                },
                {
                    'title': 'KMFDM',
                'browseId': 'UCZ2V545mkkJIeQxtCfi47NQ',
                'subscribers': '88.6K',
                'thumbnails': [{'url': 'https://lh3.googleusercontent.com/IC_M-bMWL-4toboLulY44vLMhuWgpXFfJQJ9tV9CY9CPPDYcrnUnVc2l7F9XPXIeqkrTx5vsfJK_Qw=w226-h226-l90-rj',
                    'width': 226,
                    'height': 226},
                    {'url': 'https://lh3.googleusercontent.com/IC_M-bMWL-4toboLulY44vLMhuWgpXFfJQJ9tV9CY9CPPDYcrnUnVc2l7F9XPXIeqkrTx5vsfJK_Qw=w544-h544-l90-rj',
                    'width': 544,
                    'height': 544}]}]}}

basic_keys = ['description', 'views', 'name', 'channelId', 'shuffleId', 'radioId', 'subscribers', 
    'subscribed', 'thumbnails', ('songs', 'browseId'), ('albumns', 'browseId'), ('albumns', 'params'), 
    ('singles', 'browseId'), ('singles', 'params'), ('videos', 'browseId'), ('videos', 'params'), ('related', 'browseId')]
complex_keys = [('songs', 'results'), ('albums', 'results'), ('singles', 'results'), ('videos', 'results'), ('related', 'results')]

res['albums']['results'][0]

{'title': 'CMBCRST',
 'type': '2024',
 'artists': [],
 'browseId': 'MPREb_N62W2TlZYuZ',
 'audioPlaylistId': 'OLAK5uy_kVNFc3M-jx1PPFShyAfu2qxw6QBYGaekw',
 'thumbnails': [{'url': 'https://lh3.googleusercontent.com/wImnwfNF_NejEPxBssU0H-H7kwLNodzWqiVRUZxgUT9k7KVG1YGFTbEyBQTQlZWuhpnJTuJeBmcZLlABVA=w226-h226-l90-rj',
   'width': 226,
   'height': 226},
  {'url': 'https://lh3.googleusercontent.com/wImnwfNF_NejEPxBssU0H-H7kwLNodzWqiVRUZxgUT9k7KVG1YGFTbEyBQTQlZWuhpnJTuJeBmcZLlABVA=w544-h544-l90-rj',
   'width': 544,
   'height': 544}],
 'isExplicit': True}

In [ ]:
# songs
{
    "videoId": "JLp2e2c75C0",
    "title": "Never Surrender",
    "artists": [{"name": "Combichrist", "id": "UCC4FxQ1RGHgC95YUh4nwD7w"}],
    "album": {"name": "Making Monsters", "id": "MPREb_I5yl3fkvCUu"},
    "likeStatus": "INDIFFERENT",
    "inLibrary": None,
    "thumbnails": [
        {
            "url": "https://lh3.googleusercontent.com/fw6ljhp3B3K4J-4xmspG9TvVuB_ET1RIOe3Wo4L14fwlxYBUB0mmbJWwrJBv_URwykZMSHDBtnEK-dcO=w60-h60-l90-rj",
            "width": 60,
            "height": 60,
        },
        {
            "url": "https://lh3.googleusercontent.com/fw6ljhp3B3K4J-4xmspG9TvVuB_ET1RIOe3Wo4L14fwlxYBUB0mmbJWwrJBv_URwykZMSHDBtnEK-dcO=w120-h120-l90-rj",
            "width": 120,
            "height": 120,
        },
    ],
    "isAvailable": True,
    "isExplicit": False,
    "videoType": "MUSIC_VIDEO_TYPE_ATV",
    "views": None,
}

# albums
{
    "title": "CMBCRST",
    "type": "2024",
    "artists": [],
    "browseId": "MPREb_N62W2TlZYuZ",
    "audioPlaylistId": "OLAK5uy_kVNFc3M-jx1PPFShyAfu2qxw6QBYGaekw",
    "thumbnails": [
        {
            "url": "https://lh3.googleusercontent.com/wImnwfNF_NejEPxBssU0H-H7kwLNodzWqiVRUZxgUT9k7KVG1YGFTbEyBQTQlZWuhpnJTuJeBmcZLlABVA=w226-h226-l90-rj",
            "width": 226,
            "height": 226,
        },
        {
            "url": "https://lh3.googleusercontent.com/wImnwfNF_NejEPxBssU0H-H7kwLNodzWqiVRUZxgUT9k7KVG1YGFTbEyBQTQlZWuhpnJTuJeBmcZLlABVA=w544-h544-l90-rj",
            "width": 544,
            "height": 544,
        },
    ],
    "isExplicit": True,
}



## Important fields

### search

In [3]:
query: str = "bind, torture, kill"
filter: tp.Literal["artists", "albums", "songs", "playlists"] = "songs"
limit: int = 10

youtube_client.search(query=query, filter=filter, limit=limit, ignore_spelling=False)

thumbnails = (
    [
        {
            "url": "https://lh3.googleusercontent.com/QvN2GI_GwaCFoZ1UuKVpAeOccis7Q02Lc5E5XRJ9l4rYxUa8bkmPEsJAhlqLxzkJWRFnNN7iX7-1Gdw=w60-h60-l90-rj",
            "width": 60,
            "height": 60,
        },
    ],
)


# ARTISTS
search_artists_results = [
    {
        "category": "Artists",
        "resultType": "artist",
        "artist": "Suicide Commando",
        "browseId": "UCrsvJ1npu-ox0CnncGhq2QA",
        "thumbnails": thumbnails,
    }
]

# ALBUMS
search_albums_results = [
    {
        "category": "Albums",
        "resultType": "album",
        "title": "Bind, Torture, Kill",
        "type": "Album",
        "year": "2006",
        "artists": [{"name": "Suicide Commando", "id": "UCrsvJ1npu-ox0CnncGhq2QA"}],  # artist - browseId
        "browseId": "MPREb_pRY2rME882K",
    },
]

# SONGS
search_songs_results = [
    {
        "category": "Songs",
        "resultType": "song",
        "title": "Bind, Torture, Kill",
        "album": {"name": "Bind, Torture, Kill", "id": "MPREb_pRY2rME882K"},  # album - browseId
        "videoId": "0W2JDPCnoUo",  # watch parameter music.youtube.com/watch?v=...
        "artists": [{"name": "Suicide Commando", "id": "UCrsvJ1npu-ox0CnncGhq2QA"}],  # artist - browseId
    },
]

### get_artist

In [9]:
artist_browse_id = search_artists_results[0]['browseId']
get_artist_result = youtube_client.get_artist(artist_browse_id)

# For songs/videos, pass the browseId to get_playlist().
# For albums/singles/shows, pass browseId and params to get_artist_albums().
get_artist_result_ = {
    "name": "Suicide Commando",
    "channelId": "UCrsvJ1npu-ox0CnncGhq2QA",
    "thumbnails": thumbnails,
    "songs": {
        "browseId": "VLOLAK5uy_l5rAodr7oluhYHTBYMamfhnwEeE4rmhKo",  # playlist?list=[2:]
        "results": [],  # first 5 songs shown on band page
    },
    "albums": {
        "browseId": "MPADUCrsvJ1npu-ox0CnncGhq2QA",  # use in get_artist_albums()
        "params": "ggMIegYIARoCAQI%3D",  # use in get_artist_albums()
        "results": [],  # first 10 albums shown on band page
    },
    "singles": {
        "browseId": "MPADUCrsvJ1npu-ox0CnncGhq2QA",  # use in get_artist_albums()
        "params": "ggMIegYIAhoCAQI%3D",  # use in get_artist_albums()
        "results": [],  # first 10 albums shown on band page
    },
    "videos": {
        "browseId": "VLOLAK5uy_mc_gtvS2jKs_NIeGYiCFl5PyLQN5FoMX8",  # playlist?list=[2:]
        "params": "ggMCCAI%3D",
        "results": [],  # first 10 albums shown on band page
    },
    "related": {
        "browseId": None,
        "results": [],  # 10 band recommendations
    },
}


### get_album

In [4]:
album_browse_id = search_albums_results[0]['browseId']
get_album_result = youtube_client.get_album(album_browse_id)

get_album_result_ = {
    "title": "Bind, Torture, Kill",
    "type": "Album",
    "thumbnails": thumbnails,
    "artists": [{"name": "Suicide Commando", "id": "UCrsvJ1npu-ox0CnncGhq2QA"}],  # get_artist_result_['channelId']
    "year": "2006",
    "audioPlaylistId": "OLAK5uy_klMA5C-aUaXA50Ts7QAnvSFve6PMJCOww",  # album id
    "tracks": [
        {
            "videoId": "0W2JDPCnoUo",  # watch?v=[:]&list=['audioPlaylistId]
            "title": "Bind, Torture, Kill",
            "artists": [{"name": "Suicide Commando", "id": "UCrsvJ1npu-ox0CnncGhq2QA"}],  # get_artist_result_['channelId']
            "album": "Bind, Torture, Kill",
            "videoType": "MUSIC_VIDEO_TYPE_ATV",
            "trackNumber": 1,
        }, {}, {}, {}, {}, {}, {}, {}, {}, {}, {}, 
    ],
}


### get_song

In [6]:
song_video_id = search_songs_results[0]['videoId']
get_song_result = youtube_client.get_song(song_video_id)
# get_song_result
song_video_id


'0W2JDPCnoUo'

In [ ]:
{
    "streamingData": {
        "expiresInSeconds": "21540",
        "formats": [
            {
                "itag": 18,
                "mimeType": 'video/mp4; codecs="avc1.42001E, mp4a.40.2"',
                "bitrate": 179239,
                "width": 360,
                "height": 360,
                "lastModified": "1662072309456760",
                "quality": "medium",
                "fps": 25,
                "qualityLabel": "360p",
                "projectionType": "RECTANGULAR",
                "audioQuality": "AUDIO_QUALITY_LOW",
                "approxDurationMs": "361395",
                "audioSampleRate": "44100",
                "audioChannels": 2,
                "signatureCipher": "s=AJfQdSswRAIgW9K8sePE2GtInBDtPb_mNoSNMEeqpG77k4wpbYkTbg0CIAZ1kgG8doUDXJv62aPTCK5AbM-wgU3fpy-JIPN9x1pvp5&sp=sig&url=https://rr3---sn-4g5edndl.googlevideo.com/videoplayback%3Fexpire%3D1748188680%26ei%3DqOkyaJDWIaPPi9oPxPHsgAQ%26ip%3D89.18.50.41%26id%3Do-ADJdbrLXxIu_5clEn1cfIzMkEiOSAhs5m_mrpvkKQyPj%26itag%3D18%26source%3Dyoutube%26requiressl%3Dyes%26xpc%3DEgVo2aDSNQ%253D%253D%26met%3D1748167080%252C%26mh%3DeP%26mm%3D31%252C29%26mn%3Dsn-4g5edndl%252Csn-4g5e6nsk%26ms%3Dau%252Crdu%26mv%3Dm%26mvi%3D3%26pl%3D22%26rms%3Dau%252Cau%26gcr%3Dhr%26initcwndbps%3D3350000%26bui%3DAecWEAYN1L5LUd2exjfgfibNtQkSH-Qi0P78JC7PHzbtuXIJV2RLpRCe7EGbxQVXwAadG4esXNBOtfFI%26spc%3Dwk1kZl27pIl7ImCFFnmjyxzRJpMZKiSgkHWNVhD7iGPbGFDR6xM_633kmkL8Nz-NMDCu0w%26vprv%3D1%26svpuc%3D1%26mime%3Dvideo%252Fmp4%26ns%3DY2_uJLxtLO8ZdoKbVryhs1UQ%26rqh%3D1%26cnr%3D14%26ratebypass%3Dyes%26dur%3D361.395%26lmt%3D1662072309456760%26mt%3D1748166765%26fvip%3D4%26c%3DWEB_REMIX%26sefc%3D1%26txp%3D2318224%26n%3DzKR1X8IPoJvK_kA%26sparams%3Dexpire%252Cei%252Cip%252Cid%252Citag%252Csource%252Crequiressl%252Cxpc%252Cgcr%252Cbui%252Cspc%252Cvprv%252Csvpuc%252Cmime%252Cns%252Crqh%252Ccnr%252Cratebypass%252Cdur%252Clmt%26lsparams%3Dmet%252Cmh%252Cmm%252Cmn%252Cms%252Cmv%252Cmvi%252Cpl%252Crms%252Cinitcwndbps%26lsig%3DACuhMU0wRQIhAMonRldxgRSkArAj4hiMovT4Dpu1r5RrCWeTBCM-sydDAiA2S-XgjaCLkuiJV0w-25yy5z6Os_IkohmaWqoby7E9Pw%253D%253D",
                "qualityOrdinal": "QUALITY_ORDINAL_360P",
            }
        ],
        "adaptiveFormats": [],
        "serverAbrStreamingUrl": "https://rr3---sn-4g5edndl.googlevideo.com/videoplayback?expire=1748188680&ei=qOkyaJDWIaPPi9oPxPHsgAQ&ip=89.18.50.41&id=o-ADJdbrLXxIu_5clEn1cfIzMkEiOSAhs5m_mrpvkKQyPj&source=youtube&requiressl=yes&xpc=EgVo2aDSNQ%3D%3D&met=1748167080%2C&mh=eP&mm=31%2C29&mn=sn-4g5edndl%2Csn-4g5e6nsk&ms=au%2Crdu&mv=m&mvi=3&pl=22&rms=au%2Cau&gcr=hr&initcwndbps=3350000&spc=wk1kZl24pIl7ImCFFnmjyxzRJpMYKiSgkHWNVhD7iGPbGFDR6xM_633kmkL8BzplMKg&svpuc=1&ns=kF_cibMfZatOSbB10yszjNgQ&sabr=1&rqh=1&mt=1748166765&fvip=4&keepalive=yes&c=WEB_REMIX&n=inhQMpyP9JbI9u_&sparams=expire%2Cei%2Cip%2Cid%2Csource%2Crequiressl%2Cxpc%2Cgcr%2Cspc%2Csvpuc%2Cns%2Csabr%2Crqh&sig=AJfQdSswRQIgeYxwk3PYIp5acd0EkZUxNjVw43ZclH9qGZ9SKbPeUHQCIQCZP1ZurT0q9ilsRCsK4ka0cQH8ymU-C6PRaXgjtUrqgQ%3D%3D&lsparams=met%2Cmh%2Cmm%2Cmn%2Cms%2Cmv%2Cmvi%2Cpl%2Crms%2Cinitcwndbps&lsig=ACuhMU0wRQIhAMonRldxgRSkArAj4hiMovT4Dpu1r5RrCWeTBCM-sydDAiA2S-XgjaCLkuiJV0w-25yy5z6Os_IkohmaWqoby7E9Pw%3D%3D",
    },
    "videoDetails": {
        "videoId": "0W2JDPCnoUo",
        "title": "Bind, Torture, Kill",
        "lengthSeconds": "361",
        "channelId": "UCrsvJ1npu-ox0CnncGhq2QA",
        "isOwnerViewing": False,
        "isCrawlable": True,
        "thumbnail": {
            "thumbnails": [
                {
                    "url": "https://lh3.googleusercontent.com/a3HIUTRfzxfbfJZFsn_qfP4C9Grv_sN4nzbFrXH6hLe9vMuxXPyDZ0ppr4DY32UpDZBG8Vn9xBIGHusx=w60-h60-l90-rj",
                    "width": 60,
                    "height": 60,
                },
                {
                    "url": "https://lh3.googleusercontent.com/a3HIUTRfzxfbfJZFsn_qfP4C9Grv_sN4nzbFrXH6hLe9vMuxXPyDZ0ppr4DY32UpDZBG8Vn9xBIGHusx=w120-h120-l90-rj",
                    "width": 120,
                    "height": 120,
                },
                {
                    "url": "https://lh3.googleusercontent.com/a3HIUTRfzxfbfJZFsn_qfP4C9Grv_sN4nzbFrXH6hLe9vMuxXPyDZ0ppr4DY32UpDZBG8Vn9xBIGHusx=w226-h226-l90-rj",
                    "width": 226,
                    "height": 226,
                },
                {
                    "url": "https://lh3.googleusercontent.com/a3HIUTRfzxfbfJZFsn_qfP4C9Grv_sN4nzbFrXH6hLe9vMuxXPyDZ0ppr4DY32UpDZBG8Vn9xBIGHusx=w544-h544-l90-rj",
                    "width": 544,
                    "height": 544,
                },
            ]
        },
        "allowRatings": True,
        "viewCount": "338035",
        "author": "Suicide Commando",
        "isPrivate": False,
        "isUnpluggedCorpus": False,
        "musicVideoType": "MUSIC_VIDEO_TYPE_ATV",
        "isLiveContent": False,
    },
    "microformat": {
        "microformatDataRenderer": {
            "urlCanonical": "https://music.youtube.com/watch?v=0W2JDPCnoUo",
            "title": "Bind, Torture, Kill - YouTube Music",
            "description": "Provided to YouTube by Dependent Bind, Torture, Kill · Suicide Commando Bind, Torture, Kill ℗ 2006 Dependent Released on: 2006-03-24 Main Artist: Suici...",
            "thumbnail": {
                "thumbnails": [
                    {
                        "url": "https://i.ytimg.com/vi/0W2JDPCnoUo/maxresdefault.jpg",
                        "width": 1280,
                        "height": 720,
                    }
                ]
            },
            "siteName": "YouTube Music",
            "appName": "YouTube Music",
            "androidPackage": "com.google.android.apps.youtube.music",
            "iosAppStoreId": "1017492454",
            "iosAppArguments": "https://music.youtube.com/watch?v=0W2JDPCnoUo",
            "ogType": "video.other",
            "urlApplinksIos": "vnd.youtube.music://music.youtube.com/watch?v=0W2JDPCnoUo&feature=applinks",
            "urlApplinksAndroid": "vnd.youtube.music://music.youtube.com/watch?v=0W2JDPCnoUo&feature=applinks",
            "urlTwitterIos": "vnd.youtube.music://music.youtube.com/watch?v=0W2JDPCnoUo&feature=twitter-deep-link",
            "urlTwitterAndroid": "vnd.youtube.music://music.youtube.com/watch?v=0W2JDPCnoUo&feature=twitter-deep-link",
            "twitterCardType": "player",
            "twitterSiteHandle": "@YouTubeMusic",
            "schemaDotOrgType": "http://schema.org/VideoObject",
            "noindex": False,
            "unlisted": False,
            "paid": False,
            "familySafe": True,
            "tags": ["Suicide Commando", "Bind, Torture, Kill"],
            "pageOwnerDetails": {
                "name": "Suicide Commando - Topic",
                "externalChannelId": "UCrsvJ1npu-ox0CnncGhq2QA",
                "youtubeProfileUrl": "http://www.youtube.com/channel/UCrsvJ1npu-ox0CnncGhq2QA",
            },
            "videoDetails": {
                "externalVideoId": "0W2JDPCnoUo",
                "durationSeconds": "362",
                "durationIso8601": "PT6M2S",
            },
            "linkAlternates": [
                {
                    "hrefUrl": "android-app://com.google.android.youtube/http/youtube.com/watch?v=0W2JDPCnoUo"
                },
                {"hrefUrl": "ios-app://544007664/http/youtube.com/watch?v=0W2JDPCnoUo"},
                {
                    "hrefUrl": "https://www.youtube.com/oembed?format=json&url=https%3A%2F%2Fmusic.youtube.com%2Fwatch%3Fv%3D0W2JDPCnoUo",
                    "title": "Bind, Torture, Kill",
                    "alternateType": "application/json+oembed",
                },
                {
                    "hrefUrl": "https://www.youtube.com/oembed?format=xml&url=https%3A%2F%2Fmusic.youtube.com%2Fwatch%3Fv%3D0W2JDPCnoUo",
                    "title": "Bind, Torture, Kill",
                    "alternateType": "text/xml+oembed",
                },
            ],
            "viewCount": "338035",
            "publishDate": "2021-02-25T08:54:46-08:00",
            "category": "Music",
            "uploadDate": "2021-02-25T08:54:46-08:00",
        }
    },
}

song_id = '0W2JDPCnoUo'
embed_frame = f"""
<iframe
    width="560"
    height="315"
    src="https://www.youtube.com/embed/{song_id}"
    title="YouTube video player"
    frameborder="0"
    allow="accelerometer; autoplay; clipboard-write; encrypted-media; gyroscope; picture-in-picture; web-share"
    referrerpolicy="strict-origin-when-cross-origin"
    allowfullscreen></iframe>
"""

In [36]:
youtube_client.get_artist_albums('VLOLAK5uy_mc_gtvS2jKs_NIeGYiCFl5PyLQN5FoMX8', 'ggMCCAI%3D')

KeyError: 'Unable to find \'musicCarouselShelfRenderer\' using path [\'musicCarouselShelfRenderer\', \'contents\'] on {\'musicPlaylistShelfRenderer\': {\'playlistId\': \'OLAK5uy_mc_gtvS2jKs_NIeGYiCFl5PyLQN5FoMX8\', \'header\': {\'musicSideAlignedItemRenderer\': {\'startItems\': [{\'sortFilterSubMenuRenderer\': {\'subMenuItems\': [{\'title\': \'Manual\', \'selected\': True, \'navigationEndpoint\': {\'clickTrackingParams\': \'CNAMEOPABxgAIhMIsOG30pC-jQMVH-5JBx0zewBw\', \'browseEndpoint\': {\'params\': \'ggMGKgQIABAA\', \'navigationType\': \'BROWSE_NAVIGATION_TYPE_LOAD_IN_PLACE\'}}, \'accessibility\': {\'accessibilityData\': {\'label\': \'Manual\'}}, \'trackingParams\': \'CNAMEOPABxgAIhMIsOG30pC-jQMVH-5JBx0zewBw\'}, {\'title\': \'Top voted\', \'selected\': False, \'navigationEndpoint\': {\'clickTrackingParams\': \'CM8MEOPABxgBIhMIsOG30pC-jQMVH-5JBx0zewBw\', \'browseEndpoint\': {\'params\': \'ggMGKgQIABAA\', \'navigationType\': \'BROWSE_NAVIGATION_TYPE_LOAD_IN_PLACE\'}}, \'accessibility\':